In [1]:
#import libraries
import numpy as np
import pandas
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
from time import time
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [4]:
#import data
data_small = pandas.read_csv('data64.csv')
y = data_small['label'] #original
X = data_small[data_small.columns[1:]] #original


#newly added
#reshape into 64*264 array
#data_small.iloc[3,1:].values.reshape(28, 28).astype('uint8')
#X=data_small.iloc[:,1:].values.reshape(len(data_small),28,28,1)
#y=data_small.iloc[:,0].values

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# 80-20 train-validation split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

In [5]:
#preprocess

# indices_to_keep=~X_train.isin([np.nan,np.inf,-np.inf]).any(1)

# X_train=X_train[indices_to_keep]
# y_train=y_train[indices_to_keep]

indices_to_keep=~X_training.isin([np.nan,np.inf,-np.inf]).any(1)
X_training = X_training[indices_to_keep]
y_training = y_training[indices_to_keep]

indices_to_keep=~X_validation.isin([np.nan,np.inf,-np.inf]).any(1)
X_validation = X_validation[indices_to_keep]
y_validation = y_validation[indices_to_keep]

indices_to_keep_test=~X_test.isin([np.nan,np.inf,-np.inf]).any(1)

X_test=X_test[indices_to_keep_test]
y_test=y_test[indices_to_keep_test]

#new: 
# indices_to_keep=~X.isin([np.nan,np.inf,-np.inf]).any(1)
# X = X[indices_to_keep]
# y = y[indices_to_keep]

In [6]:
#newly added
# length = len(X_train)
# X_train=X_train.iloc[:,:].values.reshape(length,64,64,1)
#y_train = y_train.values

length_train = len(X_training)
X_training=X_training.iloc[:,:].values.reshape(length_train,64,64,1)

length_validation = len(X_validation)
X_validation=X_validation.iloc[:,:].values.reshape(length_validation,64,64,1)

length_t = len(X_test)
X_test = X_test.iloc[:,:].values.reshape(length_t,64,64,1)

# length = len(X)
# X = X.iloc[:,:].values.reshape(length, 28, 28, 1)

In [7]:
X.shape

(5000, 4096)

In [8]:
X_training.shape

(3200, 64, 64, 1)

In [9]:
X_validation.shape

(800, 64, 64, 1)

In [10]:
X_test.shape

(1000, 64, 64, 1)

In [11]:
#newly added
#encoder = sklearn.preprocessing.LabelEncoder
#y_train = encoder.fit_transform(["Whole", "Half", "Quaters", "Eight", "Sixteenth"],y_train)
#print(y_train)
#y_test = keras.utils.to_categorical(y_test, num_classes=5)
y_training_n = []
for item in y_training:
    if item == "Whole":
        y_training_n.append(0)
    elif item == "Half":
        y_training_n.append(1)
    elif item == "Quarter":
        y_training_n.append(2)
    elif item == "Eight":
        y_training_n.append(3)
    elif item == "Sixteenth":
        y_training_n.append(4)
#print(y_train_n)
#y_train_n = np.array(y_train_n)
#print(y_train_n)
y_training_n = keras.utils.to_categorical(y_training_n, num_classes=5)
#print(y_train_n)

In [12]:
y_validation_n = []
for item in y_validation:
    if item == "Whole":
        y_validation_n.append(0)
    elif item == "Half":
        y_validation_n.append(1)
    elif item == "Quarter":
        y_validation_n.append(2)
    elif item == "Eight":
        y_validation_n.append(3)
    elif item == "Sixteenth":
        y_validation_n.append(4)
#print(y_train_n)
# y_train_n = np.array(y_train_n)
#print(y_train_n)
y_validation_n = keras.utils.to_categorical(y_validation_n, num_classes=5)

In [13]:
#newly added
y_test_n = []
for ele in y_test:
    if ele == "Whole":
        y_test_n.append(0)
    elif ele == "Half":
        y_test_n.append(1)
    elif ele == "Quarter":
        y_test_n.append(2)
    elif ele == "Eight":
        y_test_n.append(3)
    elif ele == "Sixteenth":
        y_test_n.append(4)
y_test_n = keras.utils.to_categorical(y_test_n, num_classes=5)
#print(y_test_n)

In [14]:
print(y_training_n.shape)
print(y_training.shape)
print(y_validation_n.shape)
print(y_validation.shape)
print(y_test_n.shape)
print(y_test.shape)
print(X_training.shape)
print(X_validation.shape)
print(X_test.shape)

(3200, 5)
(3200,)
(800, 5)
(800,)
(1000, 5)
(1000,)
(3200, 64, 64, 1)
(800, 64, 64, 1)
(1000, 64, 64, 1)


In [15]:
#CNN
model = Sequential()
model.add(Convolution2D(32,3,data_format="channels_last",activation="tanh",input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(40,11,activation="tanh"))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Convolution2D(32,3,activation='relu'))
#model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
#model.add(Dense(100))
#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 40)        154920    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 40)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4000)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 20005     
_________________________________________________________________
activation (Activation)      (None, 5)                 0

In [17]:
t0 = time()
model.fit(X_training, y_training_n, validation_data = (X_validation, y_validation_n), epochs=400)
print(time()-t0)

Epoch 1/400
100/100 [==============================] - 39s 388ms/step - loss: 1.7196 - accuracy: 0.1937 - val_loss: 1.6581 - val_accuracy: 0.2062
Epoch 2/400
100/100 [==============================] - 38s 382ms/step - loss: 1.6355 - accuracy: 0.2078 - val_loss: 1.6060 - val_accuracy: 0.2350
Epoch 3/400
100/100 [==============================] - 38s 381ms/step - loss: 1.5856 - accuracy: 0.2524 - val_loss: 1.5640 - val_accuracy: 0.2812
Epoch 4/400
100/100 [==============================] - 38s 381ms/step - loss: 1.5507 - accuracy: 0.2874 - val_loss: 1.5228 - val_accuracy: 0.3113
Epoch 5/400
100/100 [==============================] - 38s 381ms/step - loss: 1.5117 - accuracy: 0.3229 - val_loss: 1.4854 - val_accuracy: 0.3438
Epoch 6/400
100/100 [==============================] - 38s 381ms/step - loss: 1.4730 - accuracy: 0.3635 - val_loss: 1.4508 - val_accuracy: 0.3638
Epoch 7/400
100/100 [==============================] - 38s 382ms/step - loss: 1.4379 - accuracy: 0.3986 - val_loss: 1.4176 -

In [18]:
model.predict(X_test)

array([[6.7530916e-04, 1.6884597e-01, 7.7776444e-01, 5.2341163e-02,
        3.7310657e-04],
       [2.3728539e-04, 5.3497625e-04, 2.4622710e-02, 4.1608900e-01,
        5.5851597e-01],
       [9.0126169e-04, 5.7497476e-03, 4.8887908e-02, 8.3307827e-01,
        1.1138286e-01],
       ...,
       [2.9404685e-04, 4.7427122e-03, 5.3488585e-04, 1.1714450e-01,
        8.7728393e-01],
       [1.5839419e-04, 8.6387074e-01, 1.2885812e-01, 6.8705832e-03,
        2.4209051e-04],
       [5.7866634e-03, 4.3339437e-01, 4.9995178e-01, 5.7176523e-02,
        3.6906975e-03]], dtype=float32)

In [19]:
model.evaluate(X_test, y_test_n)

32/32 [==============================] - 3s 75ms/step - loss: 0.2049 - accuracy: 0.9600


[0.2049170434474945, 0.9599999785423279]